In [8]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os 
import numpy as np
from tensorflow import keras
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from methods_audio import data_handling
from methods_audio import data_augmentation
from methods_audio import denoising 
from methods_audio import model_performance_training
from methods_audio import model_training



# 1. Get data (file names)

In [17]:
data = data_handling.get_data()

In [18]:
type(data)

tensorflow.python.data.ops.shuffle_op._ShuffleDataset

# 2. Read data (transforming file names into waves) <br>
Additionally, the mean is removed and the data is normalized. 

In [12]:
data = data.map(data_handling.read_in_data) 

# 3. Get input for model training 

In [6]:
samples, labels = data_handling.extract_samples_labels(data)

2023-05-06 18:51:25.140497: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


# 4. Split data into train and validation sets

In [ ]:
validation_set_size = 0.30
x_train, x_valid, y_train, y_valid = train_test_split(samples, labels, test_size= validation_set_size, random_state=123)

# 5. Data Augmentation (on train set only)

In [ ]:
# Define type of augmentation to do. Options: signal, spectogram, both, none 
type_augmentation = 'signal'

In [ ]:
if (type_augmentation == 'signal'):
    x_train, y_train = data_augmentation.time_augmentation(x_train, y_train)
elif(type_augmentation == 'spectrogram'):
    # TODO: work out what to do  
    x_train, y_train = data_augmentation.spectrogram_augmentaion(x_train, y_train)
elif(type_augmentation == 'both'): 
    # TODO: sort this
    pass 

# 6. Denoising (on train and validation set)

In [ ]:
# Define type of denoising to do. Options: spectral_gating, low_pass, none 
type_denoising= 'spectral'
differentiation = True 

# For low pass filter:
low_pass_cutoff = 500 # this value will be used for experimentation, ranging from 500-4000. 
low_pass_order = 4 # this value will be used for experimentation 

In [ ]:
if (type_denoising== 'spectral'): 
    denoising.apply_spectral(x_train)
    denoising.apply_spectral(x_valid)
elif(type_denoising == 'low_pass'): 
    denoising.apply_low_pass(x_train, cutoff, order)
    denoising.apply_low_pass(x_valid, cutoff, order)

# 7. Transform data into Spectogram/Mel-spectogram 

In [ ]:
# Define type of transformation to do. Options = spectrogram, mel_spectrogram, db_mel_spectrogram
type_transformation = 'spectrogram' 

In [ ]:
x_train = data_handling.transform_data(x_train, type_transformation)
x_valid = data_handling.transform_data(x_valid, type_transformation)

# 8. Upload model

In [ ]:
# Select which model to use: 
model_name = '01'
location_model = 'data/models/' + model_name 
location_weights = 'data/models/weights' + model_name

batch_size = 8 
epoch = 20 

In [ ]:
if (model_name == '01'):
    learning_rate = 0.02661877777328162 # result from param optimization for Model 1 
elif (model_name == '02'): 
    learning_rate = 0.01

In [ ]:
model = keras.models.load_model(location_model)
model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=learning_rate), 
    loss="BinaryCrossentropy", 
    metrics = ['accuracy', 'Recall', 'Precision'],
    )

# 9. Evaluate performance of the model by training it with K-Fold Validation

In [ ]:
confusion_matrices, histories = model_performance_training.train_performance_k_fold(location_model, samples, labels, learning_rate, epoch, batch_size,type_augmentation, type_denoising, low_pass_cutoff, low_pass_order, type_transformation)


# 10. If any certain model perform well, it can be trained and exported as final model using: 

In [ ]:
#model, history = model_training(model, x_train, y_train, x_valid, y_train, batch_size, epoch)
#model.save_weights(location_weights, overwrite = True, save_format = 'tf', options = None) # saving as TensorFlow format 